In [1]:
import pandas as pd
import random
import numpy as np

from bs4 import BeautifulSoup
import requests
import re
import json
import csv

from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select 

from time import sleep

pd.set_option('display.max_columns', None)


C:\Users\Isabel\AppData\Local\Temp\ipykernel_13288\3175742525.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
with open('../Fase1_datos/lista_accion.json', 'r') as fichero:
    accion = json.load(fichero)

In [3]:
len(accion)

3679

In [20]:
busqueda = accion[505:1000]
len(busqueda)

495

In [5]:
# variables donde iremos almacenando todo
info_por_peli = []
fallos = []

In [21]:
# llegar a web:
driver = webdriver.Chrome()
driver.get('https://www.imdb.com/')
driver.maximize_window()
sleep(random.randint(1,3))


# rechazar cookies
driver.find_element('css selector', '#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-hLBbgP.sc-ftTHYK.dcvrLS.dufgkr.ecppKW').click()
sleep(random.randint(1,3))

contador = 0

# entrando en cada título
for peli in busqueda:
    lista_info = [f'{peli[0]}',]
    contador +=1
    print(f'Película: {contador}')

    try:
        
        # metemos el ID en el buscador
        try:
            driver.find_element('css selector', '#suggestion-search').send_keys(peli[0], Keys.ENTER)
            sleep(random.randint(1,3))

            # extraemos info
            info = driver.find_element('xpath' , f'//*[@id="__next"]/main/div/section[1]/section/div[3]/section').text
            lista_info.append(info.split('\n'))

            info_por_peli.append(lista_info)
        except:
            fallos.append(peli)
            driver.find_element('css selector', '#error > div.error_attrib > a').click()
        
                    
    except:
        continue

Película: 1
Película: 2
Película: 3
Película: 4
Película: 5
Película: 6
Película: 7
Película: 8
Película: 9
Película: 10
Película: 11
Película: 12
Película: 13
Película: 14
Película: 15
Película: 16
Película: 17
Película: 18
Película: 19
Película: 20
Película: 21
Película: 22
Película: 23
Película: 24
Película: 25
Película: 26
Película: 27
Película: 28
Película: 29
Película: 30
Película: 31
Película: 32
Película: 33
Película: 34
Película: 35
Película: 36
Película: 37
Película: 38
Película: 39
Película: 40
Película: 41
Película: 42
Película: 43
Película: 44
Película: 45
Película: 46
Película: 47
Película: 48
Película: 49
Película: 50
Película: 51
Película: 52
Película: 53
Película: 54
Película: 55
Película: 56
Película: 57
Película: 58
Película: 59
Película: 60
Película: 61
Película: 62
Película: 63
Película: 64
Película: 65
Película: 66
Película: 67
Película: 68
Película: 69
Película: 70
Película: 71
Película: 72
Película: 73
Película: 74
Película: 75
Película: 76
Película: 77
Película

In [22]:
len(info_por_peli)

989

In [23]:
len(fallos)

4

In [24]:
with open('FallosComedia.json', 'w') as fichero:
    json.dump(fallos, fichero)

In [25]:
lista_tuplas = []

In [26]:
for peli in info_por_peli:
    
    patron = re.compile(r'([a-z])([A-Z])')
    patron_duracion = re.compile(r'\b(?:\d+h\s*)?\d+min\b')
    tupla_datos = [peli[0],] 

    # listas intermedias para ir checkeando
    direccion = []
    guion = []
    nota = []
    duracion = []
    argumento = []
        
    for indice, linea in enumerate(peli[1]):
               
        
        # directores    
        if 'Dirección' in linea:
                                
            directores = patron.sub(r'\1, \2', peli[1][indice+1])
            direccion.append(directores)            


        # guionistas
        elif 'Guión' in linea:
            
            guionistas = patron.sub(r'\1, \2', peli[1][indice+1])
            guion.append(guionistas)
                
        
        # nota y duracion
        elif 'PUNTUACIÓN EN IMDb' in linea:
            nota.append(float(peli[1][indice+1].replace(',', '.')))
            
            if r'\d+h' or r'\d+min' in peli[1][indice-1]:  
                duracion.append(peli[1][indice-1])
            
                    
        # argumento
        elif 'Añade un argumento en tu idioma' in linea:
            argumento.append(peli[1][indice+1])

    # añadimos a la tupla los datos de la lista intermedia
    for elemento in [direccion, guion, nota, duracion, argumento]:
        if elemento != []:
            tupla_datos.append(elemento[0])
        else:
            tupla_datos.append(np.nan)
    
    lista_tuplas.append(tupla_datos)

In [27]:
len(lista_tuplas)

989

In [19]:
with open('AccionWIP.json', 'w') as fichero:
    json.dump(lista_tuplas, fichero)

In [10]:
len(fallos)

0